In [1]:
from __future__ import print_function
import os

import numpy as np
import zipfile
import tarfile
from six.moves.urllib.request import urlretrieve
import shutil 
import random

import string
import tensorflow as tf

# Dirs - must be absolute paths!
LOG_DIR = '/tmp/tf/ptb_char_lstm_mann_lrua_shared_memory_unified_prev/h32b10s1mem10lr2/'
# Local dir where PTB files will be stored.
PTB_DIR = '/home/tkornuta/data/ptb/'

# Filenames.
TRAIN = "ptb.train.txt"
VALID = "ptb.valid.txt"
TEST = "ptb.test.txt"

# Size of the hidden state
HIDDEN_SIZE = 32

# Size of batch
BATCH_SIZE = 10

# Length of sequence (=  number of units of controller (recurrent layer))
SEQ_LENGTH = 1

#### MANN-related parameters.
# Size of the local memory of each cell.
MEMORY_SLOTS = 10

# Number of smallest elements - used in LRUA scheme.
N_SMALLEST = 1

# "Update weight decay".
GAMMA = 1e-4 

# Eps for normalization in visualization
EPS = 1e-10

In [2]:
def maybe_download_ptb(path, 
                       filename='simple-examples.tgz', 
                       url='http://www.fit.vutbr.cz/~imikolov/rnnlm/', 
                       expected_bytes =34869662):
  # Eventually create the PTB dir.
  if not tf.gfile.Exists(path):
    tf.gfile.MakeDirs(path)
  """Download a file if not present, and make sure it's the right size."""
  _filename = path+filename
  if not os.path.exists(_filename):
    print('Downloading %s...' % filename)
    _filename, _ = urlretrieve(url+filename, _filename)
  statinfo = os.stat(_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', (_filename), '(', statinfo.st_size, ')')
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + _filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download_ptb(PTB_DIR)

Found and verified /home/tkornuta/data/ptb/simple-examples.tgz ( 34869662 )


### Check/maybe download PTB.

### Extract dataset-related files from the PTB archive.

In [3]:
def extract_ptb(path, filename='simple-examples.tgz', files=["ptb.train.txt", "ptb.valid.txt", "ptb.test.txt", 
                                       "ptb.char.train.txt", "ptb.char.valid.txt", "ptb.char.test.txt"]):
    """Extracts files from PTB archive."""
    # Extract
    tar = tarfile.open(path+filename)
    tar.extractall(path)
    tar.close()
    # Copy files
    for file in files:
        shutil.copyfile(PTB_DIR+"simple-examples/data/"+file, PTB_DIR+file)
    # Delete directory
    shutil.rmtree(PTB_DIR+"simple-examples/")        

extract_ptb(PTB_DIR)

### Load train, valid and test texts.

In [4]:
def read_data(filename, path):
    with open(path+filename, 'r') as myfile:
        data=myfile.read()# .replace('\n', '')
        return data

train_text = read_data(TRAIN, PTB_DIR)
train_size=len(train_text)
print(train_size, train_text[:100])

valid_text = read_data(VALID, PTB_DIR)
valid_size=len(valid_text)
print(valid_size, valid_text[:64])

test_text = read_data(TEST, PTB_DIR)
test_size=len(test_text)
print(test_size, test_text[:64])

5101618  aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memote
399782  consumers may want to move their telephones a little closer to 
449945  no it was n't black monday 
 but while the new york stock excha


### Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = 59 # [A-Z] + [a-z] + ' ' +few 'in between; + punctuation
first_letter = ord(string.ascii_uppercase[0]) # ascii_uppercase before lowercase! 
print("vocabulary size = ", vocabulary_size)
print(first_letter)

def char2id(char):
  """ Converts char to id (int) with one-hot encoding handling of unexpected characters"""
  if char in string.ascii_letters:# or char in string.punctuation or char in string.digits:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    # print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  """ Converts single id (int) to character"""
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

#print(len(string.punctuation))
#for i in string.ascii_letters:
#    print (i, char2id(i))


print(char2id('a'), char2id('A'), char2id('z'), char2id('Z'), char2id(' '), char2id('ï'))
print(id2char(char2id('a')), id2char(char2id('A')))
#print(id2char(65), id2char(33), id2char(90), id2char(58), id2char(0))
#bankno
sample = np.zeros(shape=(1, vocabulary_size), dtype=np.float)
sample[0, char2id(' ')] = 1.0
print(sample)

vocabulary size =  59
65
33 1 58 26 0 0
a A
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]]


### Helper class for batch generation

In [6]:
class BatchGenerator(object):
  def __init__(self, text, batch_size, seq_length, vocab_size):
    """
    Initializes the batch generator object. Stores the variables and first "letter batch".
    text is text to be processed
    batch_size is size of batch (number of samples)
    seq_length represents the length of sequence
    vocab_size is number of words in vocabulary (assumes one-hot encoding)
    """
    # Store input parameters.
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._seq_length = seq_length
    self._vocab_size = vocab_size
    # Divide text into segments depending on number of batches, each segment determines a cursor position for a batch.
    segment = self._text_size // batch_size
    # Set initial cursor position.
    self._cursor = [ offset * segment for offset in range(batch_size)]
    # Store first "letter batch".
    self._last_letter_batch = self._next_letter_batch()
  
  def _next_letter_batch(self):
    """
    Returns a batch containing of encoded single letters depending on the current batch 
    cursor positions in the data.
    Returned "letter batch" is of size batch_size x vocab_size
    """
    letter_batch = np.zeros(shape=(self._batch_size, self._vocab_size), dtype=np.float)
    # Iterate through "samples"
    for b in range(self._batch_size):
      # Set 1 in position pointed out by one-hot char encoding.
      letter_batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return letter_batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    # First add last letter from previous batch (the "additional one").
    batches = [self._last_letter_batch]
    for step in range(self._seq_length):
      batches.append(self._next_letter_batch())
    # Store last "letter batch" for next batch.
    self._last_letter_batch = batches[-1]
    return batches


In [7]:
# Trick - override first 10 chars
#list1 = list(train_text)
#for i in range(2):
#    list1[i] = 'z'
#train_text = ''.join(list1)
#print("Train set =", train_text[0:100])

# Create objects for training, validation and testing batch generation.
train_batches = BatchGenerator(train_text, BATCH_SIZE, SEQ_LENGTH, vocabulary_size)

# Get first training batch.
#batch = train_batches.next()
#print(len(batch))
#print(batch[0].shape)
#print("Batch = ", batch)
#print(batches2string(batch))
#print("batch len = num of enrollings",len(batch))
#for i in range(num_unrollings):
#    print("i = ", i, "letter=", batches2string(batch)[0][i][0], "bits = ", batch[i][0])


# For validation  - process the whole text as one big batch.
VALID_BATCH_SIZE = int(np.floor(valid_size/SEQ_LENGTH))
valid_batches = BatchGenerator(valid_text, VALID_BATCH_SIZE, SEQ_LENGTH, vocabulary_size)
# Get single batch! 
valid_batch = valid_batches.next()

# For texting  - process the whole text as one big batch.
TEST_BATCH_SIZE = int(np.floor(test_size/SEQ_LENGTH))
test_batches = BatchGenerator(test_text, TEST_BATCH_SIZE, SEQ_LENGTH, vocabulary_size)
# Get single batch! 
test_batch = test_batches.next()


### Helper functions - used during graph definition

In [8]:
# Function adding visualization to a given "matrix" (keys, memory etc.) with additional normalization
def visualize_hot_cold_normalized(matrix_,
                    axis_of_reduction_,
                    name_):
        # Create hot-cold visualization (red=positive/blue=negative)
        zeros = tf.zeros_like(matrix_) 
        
        # Get negative values only.
        neg = tf.less(matrix_, zeros)
        blue = tf.multiply(tf.cast(neg, tf.float32), matrix_)
        min_blue = tf.reduce_min(matrix_, axis=axis_of_reduction_, keep_dims=True) + EPS
        norm_blue = 255.0 * blue/min_blue
        
        # Get positive values only.
        pos = tf.greater(matrix_, zeros)
        red = tf.multiply(tf.cast(pos, tf.float32), matrix_)
        max_red = tf.reduce_max(matrix_, axis=axis_of_reduction_, keep_dims=True) + EPS
        norm_red = 255.0 * red/max_red
        
        # Stack them into three channel image with hot-cold values.
        rgb = tf.stack([norm_red, zeros, norm_blue], axis=2)

        #print("name_=",name_)
        #print("np.int32(zeros.shape[1]=", np.int32(zeros.shape[1]))
        # TODO: find fix and get rid of the batch_size(!)
        rgb_reshaped = tf.reshape(rgb, [1, -1,  np.int32(zeros.shape[1]), 3])

        # Visualize read weights as image.
        rgb_reshaped_summary = tf.summary.image(name_+"_normalized", rgb_reshaped)

In [9]:
# Function adding visualization to a given "matrix" (keys, memory etc.)
def visualize_hot_cold(matrix_,
                    name_):
        # Create hot-cold visualization (red=positive/blue=negative)
        zeros = tf.zeros_like(matrix_) 
        
        # Get negative values only.
        neg = tf.less(matrix_, zeros)
        blue = 255.0 * tf.multiply(tf.cast(neg, tf.float32), matrix_)
        
        # Get positive values only.
        pos = tf.greater(matrix_, zeros)
        red = 255.0 * tf.multiply(tf.cast(pos, tf.float32), matrix_)
        
        # Stack them into three channel image with hot-cold values.
        rgb = tf.stack([red, zeros, blue], axis=2)

        #print("name_=",name_)
        #print("np.int32(zeros.shape[1]=", np.int32(zeros.shape[1]))
        # TODO: find fix and get rid of the batch_size(!)
        rgb_reshaped = tf.reshape(rgb, [1, -1,  np.int32(zeros.shape[1]), 3])

        # Visualize read weights as image.
        rgb_reshaped_summary = tf.summary.image(name_, rgb_reshaped)


In [10]:
# Definition of the controller cell graph.
def controller_cell(input_, # input x
                    memory_input_, # read vector from the memory returned by previous cell
                    prev_output_, # output of the previous cell
                    prev_cell_state_, # previous cell state
                    prev_read_weights_, # read weights from previous time state (t-1) [1xMEMORY_SLOTS]
                    prev_update_weights_, # update weights from previous time state (t-1) [1xMEMORY_SLOTS]
                    name_):
    """Create a controller with local memory cell"""
    """First dimensions of each of the computational nodes below is "derrived" from BATCH_SIZE"""
    with tf.name_scope(name_):

        with tf.name_scope("LSTM"):
            # LSTM cell equations according to Christopher Olah blog.
            # colah.github.io/posts/2015-08-Understanding-LSTMs/
            # Concatenate intyp x with h_prev ("prev output") TODO: and memory.
            i_h_m = tf.concat([input_, prev_output_, memory_input_], 1)

            # Calculate forget, input and output gate activations.
            forget_gate = tf.sigmoid(tf.matmul(i_h_m, cWf) + cbf, name="Forget_gate")
            input_gate = tf.sigmoid(tf.matmul(i_h_m, cWi) + cbi, name="Input_gate")
            output_gate = tf.sigmoid(tf.matmul(i_h_m, cWo) + cbo, name="Output_gate")

            # Update of the cell state C~.
            cell_update = tf.tanh(tf.matmul(i_h_m, cWc) + cbc, name="Cell_update")
            # New cell state C.
            cell_state = tf.add(forget_gate * prev_cell_state_, input_gate * cell_update, name = "Cell_state")
            # Calculate h - "output".
            cell_output = output_gate * tf.tanh(cell_state)
            
        with tf.name_scope("Keys"):
            # Calculate keys - read and add.
            #k_t = tf.tanh(tf.matmul(cell_output, W_key) + b_key) # (batch_size, nb_reads, memory_size[1])
            #a_t = tf.tanh(tf.matmul(cell_output, W_add) + b_add) # (batch_size, nb_reads, memory_size[1])
            #alpha = tf.sigmoid(tf.matmul(cell_output, W_alpha) + b_alpha) # (batch_size, nb_reads, 1)
            # Latest vs LRU ratio.
            alpha = tf.Variable(tf.truncated_normal(shape=[]), name="Alpha")

            # Add histograms to TensorBoard.
            #k_t_hist = tf.summary.histogram("k_t", k_t)
            #a_t_hist = tf.summary.histogram("a_t", a_t)
            alpha_scalar = tf.summary.scalar("alpha", alpha)
            
        # Read from the memory.
        with tf.name_scope("Read_head"):
            
            # Normalize batch.
            norm_batch = tf.nn.l2_normalize(cell_output,1)
            # Normalize memory.
            norm_memory = tf.nn.l2_normalize(memory,0)
            # Calculate cosine similarity.
            #similarity_batch = tf.matmul(norm_batch, norm_memory) 

            # Not-normalized similarity with min value adjustment.
            similarity_batch = tf.matmul(cell_output, memory) 
            #sim_min_batch = tf.reduce_min(similarity_batch, axis=1)
            #similarity_batch_adj = similarity_batch - sim_min_batch # neg values will become 0.
            
            # Calculate read weights based on similarity.
            read_weights_batch = tf.nn.softmax(10 *similarity_batch)

            # Add to list returned as "previous read weights" - reduced to [1xMEMORY_SLOTS] for the whole batch!
            read_weights_seq.append(tf.reduce_mean(read_weights_batch, axis=0, keep_dims=True))
            
            # Add histograms to TensorBoard.
            norm_batch_hist = tf.summary.histogram("norm_batch", norm_batch)
            norm_batch_hist = tf.summary.histogram("norm_batch", norm_batch)
            norm_memory_hist = tf.summary.histogram("norm_memory", norm_memory)
            similarity_batch_hist = tf.summary.histogram("cosine_similarity_batch", similarity_batch)
            read_weights_batch_hist = tf.summary.histogram("read_weights_batch", read_weights_batch)
            
            # Add hot-cold visualizations.
            visualize_hot_cold_normalized(matrix_=cell_output, axis_of_reduction_=None, name_="cell_output")          
            #hot_cold_visualization(matrix_=k_t, axis_of_reduction_=1, name_="k_t")          
            visualize_hot_cold_normalized(matrix_=similarity_batch, axis_of_reduction_=1, name_="similarity_batch")          
            #visualize_hot_cold_normalized(matrix_=similarity_batch_adj, axis_of_reduction_=1, name_="similarity_batch_adj")          
            visualize_hot_cold(matrix_=read_weights_batch, name_="read_weights_batch")          
        
        with tf.name_scope("Memory_output"):
            # Calcualte read vector.
            memory_output_batch_without_dims = tf.tensordot(read_weights_batch, tf.transpose(memory), axes=1, name="Memory_output_batch_r")   
            # TF bugfix - tensordot not returning proper shapes when only partial shapes are known. :]
            # https://github.com/tensorflow/tensorflow/issues/6682
            memory_output_batch = tf.reshape(memory_output_batch_without_dims, [-1, HIDDEN_SIZE])
            # Add histograms to TensorBoard.
            memory_output_batch_hist = tf.summary.histogram("memory_output_batch", memory_output_batch)
            # Add hot-cold visualizations.
            visualize_hot_cold_normalized(matrix_=memory_output_batch, axis_of_reduction_=None, name_="memory_output_batch")          

        with tf.name_scope("Write_head"):
            # "Truncation scheme to update the least-used positions".
            # First, find (size-n) top elements (one for all samples in a batch, but separatelly for each cell).
            top = tf.nn.top_k(-prev_update_weights_, N_SMALLEST)
            # To get boolean True/False values, you can first get the k-th value and then use tf.greater_equal:
            kth = tf.reduce_min(top.values, axis=1, keep_dims=True)
            top2 = tf.greater_equal(-prev_update_weights_, kth)
            # And finally - cast it to n smallest elements.
            prev_smallest_lru_weights = tf.cast(top2, tf.float32)

            #write_weights_seq_batch.append(prev_smallest_lru_weights)
            write_weights = tf.add(tf.sigmoid(alpha) * prev_read_weights_,
                                   (1.0 - tf.sigmoid(alpha)) * prev_smallest_lru_weights,
                                   name="Write_weights_ww")

            # Add histograms to TensorBoard.
            prev_smallest_lru_weights_hist = tf.summary.histogram("prev_smallest_lru_weights", prev_smallest_lru_weights)
            write_weights_hist = tf.summary.histogram("write_weights", write_weights)

            # Add hot-cold visualizations.
            visualize_hot_cold_normalized(matrix_=write_weights, axis_of_reduction_=None, name_="write_weights")          

        with tf.name_scope("Update_head"):
            # This relies on prev. weights and will be used in fact in the NEXT step.
            rw_sum_reduced = tf.reduce_mean(read_weights_batch, axis=0, keep_dims=True) + write_weights
            update_weights = tf.add(GAMMA * prev_update_weights_,
                                            rw_sum_reduced,
                                            name="Update_weights_uw")
            
            # Add to list returned as "previous update weights".
            update_weights_seq.append(update_weights)
            # Add histograms to TensorBoard.
            update_weights_hist = tf.summary.histogram("update_weights", update_weights)
            # Add hot-cold visualizations.
            visualize_hot_cold_normalized(matrix_=update_weights, axis_of_reduction_=None, name_="update_weights")          

            
    with tf.name_scope("Memory_update"):
        # Perform single update for each sequence/batch.
        #memory_update_without_dims = tf.tensordot(cell_output_sum, write_weights, axes=1)
        # TF bugfix - tensordot not returning proper shapes when only partial shapes are known. :]
        # https://github.com/tensorflow/tensorflow/issues/6682
        #memory_update = tf.reshape(memory_update_without_dims, [HIDDEN_SIZE, MEMORY_SLOTS]) 
        
        cell_output_sum_transposed = tf.transpose(tf.reduce_sum(cell_output, axis=0, keep_dims=True))
        memory_update = cell_output_sum_transposed * write_weights
        memory_update_seq.append(memory_update)
        
        # Add dependendency control - first prediction?
        #with tf.control_dependencies([prediction_batch]):
        # Update the memory.
        memory_updated = memory + memory_update
        
        # Memory "truncation".
        #memory_trunk_op = memory.assign(memory_updated)
        #memory_trunk_op = memory.assign(tf.tanh(memory_updated))
        memory_trunk_op = memory.assign(tf.minimum(tf.maximum(memory_updated, -1), 1))

        # Add histograms to TensorBoard.
        memory_update_hist = tf.summary.histogram("memory_update", memory_update)
        memory_hist = tf.summary.histogram("memory_before_truncation", memory_updated)
        memory_trunk_hist = tf.summary.histogram("memory_after_truncation", memory_trunk_op)

        # Add hot-cold visualizations.
        visualize_hot_cold_normalized(matrix_=memory_update, axis_of_reduction_=None, name_="memory_update")          
        visualize_hot_cold_normalized(matrix_=memory_updated, axis_of_reduction_=None, name_="memory_before_truncation")          
        visualize_hot_cold_normalized(matrix_=memory, axis_of_reduction_=None, name_="memory_after_truncation")          
        
    return memory_output_batch, cell_output, cell_state

print("Cell definition OK")

Cell definition OK


###  Definition of tensor graph

In [11]:
# Reset graph - just in case.
tf.reset_default_graph()

# Memory.
memory = tf.Variable(tf.truncated_normal(shape=[HIDDEN_SIZE, MEMORY_SLOTS]), trainable=False, name="Memory_M")

# 0. Previous variables.
with tf.name_scope("Previous_variables"):

    # Placeholders for previous weights.
    # Each cell (element in sequence) has its own prev read/update vector - ONE FOR THE WHOLE BATCH [1xMEMORY_SIZE]
    prev_read_weights_seq = list()    
    prev_update_weights_seq = list()    
    for i_seq in range(SEQ_LENGTH):
        prev_read_weights_seq.append(tf.placeholder(tf.float32, shape=[1, MEMORY_SLOTS], name="Prev_rw"))
        prev_update_weights_seq.append(tf.placeholder(tf.float32, shape=[1, MEMORY_SLOTS], name="Prev_uw"))

# 1. Placeholders for inputs.
with tf.name_scope("Input_data"):
    # Define input data buffers.
    data_buffers = list()
    for _ in range(SEQ_LENGTH + 1):
        # Collect placeholders for inputs/labels: Batch x Vocab size.
        data_buffers.append(tf.placeholder(tf.float32, shape=[None, vocabulary_size], name="data_buffers"))

    # Sequence of batches.
    input_seq_batch = data_buffers[:SEQ_LENGTH]

    # Labels are pointing to the same placeholders!
    # Labels are inputs shifted by one time step.
    labels_seq_batch = data_buffers[1:]  
    # Concatenate targets into 2D tensor.
    target_batch = tf.concat(labels_seq_batch, 0)

    # Add histograms to TensorBoard.
    input_seq_batch_hist = tf.summary.histogram("input_seq_batch", input_seq_batch)

# 2. Unrolled controller ops.
with tf.name_scope("Controller"):
    # Define controller parameters:
    # Input gate: input, previous output, and bias.
    cWf = tf.Variable(tf.truncated_normal([vocabulary_size+2*HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="cWf")
    cbf = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bf")

    # Forget gate: input, previous output, and bias.
    cWi = tf.Variable(tf.truncated_normal([vocabulary_size+2*HIDDEN_SIZE,HIDDEN_SIZE], -0.1, 0.1), name="cWi")
    cbi = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bi")

    # Output gate: input, previous output, and bias.
    cWo = tf.Variable(tf.truncated_normal([vocabulary_size+2*HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="cWo")
    cbo = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bo")

    # Memory cell: input, state and bias.                             
    cWc = tf.Variable(tf.truncated_normal([vocabulary_size+2*HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="cWc")
    cbc = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="bc")

    # Read key.
    #W_key = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="W_key")
    #b_key = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="b_key")
    
    # Add key.
    #W_add = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, HIDDEN_SIZE], -0.1, 0.1), name="W_add")
    #b_add = tf.Variable(tf.zeros([1, HIDDEN_SIZE]), name="b_add")
    
    # Alpha - used in Latest vs LRU ratio.
    #W_alpha = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, 1], -0.1, 0.1), name="W_alpha")
    #b_alpha = tf.Variable(tf.zeros([1, 1]), name="b_alpha")
    
    # Placeholders for "zero" (the oldest) state and output: Batch x Hidden size.
    init_controller_output = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_controller_output")
    init_controller_state = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_controller_state")
    # Placeholder for "zero" memory read: Batch X Hidden (TODO: memory?) size.
    init_memory_output = tf.placeholder(tf.float32, shape=[None, HIDDEN_SIZE], name="init_memory_read")

    # Unrolled LSTM.
    # Build outpus of size SEQ_LENGTH.
    controller_outputs_batch_seq = list()
    memory_outputs_batch_seq = list()
    # Two lists that will be "returned" and later passed as previous states. 
    read_weights_seq = list()  
    update_weights_seq = list()  
    memory_update_seq = list()
    
    # "Link" oldest statte and output to placeholders.
    controller_output = init_controller_output
    controller_state = init_controller_state
    memory_output = init_memory_output
    # For every buffer in input sequence batch buffers...
    for i in range(SEQ_LENGTH):
        # ... add cell...     
        memory_output, controller_output, controller_state = controller_cell(
            input_seq_batch[i], 
            memory_output, 
            controller_output, 
            controller_state, 
            prev_read_weights_seq[i],
            prev_update_weights_seq[i],
            "cell_"+str(i))
        # .. and add controller buffer to outputs.
        controller_outputs_batch_seq.append(controller_output)
        memory_outputs_batch_seq.append(memory_output)
         
    # Add histograms to TensorBoard.
    controller_outputs_batch_seq_hist = tf.summary.histogram("controller_outputs_batch_seq", controller_outputs_batch_seq)
    memory_outputs_batch_seq_hist = tf.summary.histogram("memory_outputs_batch_seq", memory_outputs_batch_seq)
    memory_hist = tf.summary.histogram("memory", memory)
    read_weights_seq_hist = tf.summary.histogram("read_weights_seq", read_weights_seq)
    update_weights_seq_hist = tf.summary.histogram("update_weights_seq", update_weights_seq)

# 3. Output ops.
with tf.name_scope("Output"):
    # Concatenate controller hidden state with the read vector.
    concat_batch_seq = list()    
    for i_seq in range(SEQ_LENGTH):
        concat_output_batch = tf.concat([controller_outputs_batch_seq[i_seq],
                                        memory_outputs_batch_seq[i_seq]],
                                        axis=1, 
                                        name="Concat_mann_output")
        concat_batch_seq.append(concat_output_batch)   
        # Add hot-cold visualizations.
        visualize_hot_cold_normalized(matrix_=concat_output_batch, axis_of_reduction_=None, name_="concatenated_mann_output_batch")          

    # Add histograms to TensorBoard.
    concat_batch_seq_hist = tf.summary.histogram("concat_batch_seq", concat_batch_seq)
    
    #output_batch = tf.concat(memory_outputs_batch_seq, 0) 
    output_batch = tf.concat(concat_batch_seq, 0)
 
    # Output layer weights and biases.
    Wh = tf.Variable(tf.truncated_normal([2*HIDDEN_SIZE, vocabulary_size], -0.1, 0.1), name="Wh")
    bh = tf.Variable(tf.zeros([vocabulary_size]), name="Wh")

    # Logits.
    logits_batch = tf.nn.xw_plus_b(output_batch, Wh, bh, name = "Logits")
    # Add fully connected softmax layer on top - predictions.
    prediction_batch = tf.nn.softmax(logits_batch)

 
# 4. Loss ops.
with tf.name_scope("Loss"):
    # Loss function(s) - one for every output generated by every LSTM cell.
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=target_batch, logits=logits_batch))
    # Add loss summary.
    loss_summary = tf.summary.scalar("loss", loss)

# 5. Training ops.  
with tf.name_scope("Optimization"):
    # Learning rate decay.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(0.1, global_step, 5000, 0.9, staircase=True)
    # Optimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)
    gradients, v = zip(*grads_and_vars)
    # Gradient clipping.
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

# Merge all summaries.
merged_summaries = tf.summary.merge_all()

print("Graph definition OK")

Graph definition OK


In [12]:
def create_feed_dict(set_type_):
    """Creates a dictionaries for different sets: maps data onto Tensor placeholders."""
    feed_dict = dict()
    
    if set_type_=="train":
        # Get next batch and create a feed dict.
        next_batch = train_batches.next()
        # Feed batch to input buffers.
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = next_batch[i]

        # Set previous weights of read and write heades.
        for i in range(SEQ_LENGTH):
            feed_dict[prev_read_weights_seq[i]] = prev_rw_seq[i]
            feed_dict[prev_update_weights_seq[i]] = prev_uw_seq[i]

        # Reset "init" state and output of controller.
        feed_dict[init_controller_output] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
        feed_dict[init_controller_state] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
        feed_dict[init_memory_output] = np.zeros([BATCH_SIZE, HIDDEN_SIZE])
            
    elif set_type_=="valid":
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = valid_batch[i]

        # Set previous weights of read and write heades.
        for i in range(SEQ_LENGTH):
            feed_dict[prev_read_weights_seq[i]] = prev_rw_seq[i]
            feed_dict[prev_update_weights_seq[i]] = prev_uw_seq[i]

        # Reset "init" state and output of controller.
        feed_dict[init_controller_output] = np.zeros([valid_size, HIDDEN_SIZE])
        feed_dict[init_controller_state] = np.zeros([valid_size, HIDDEN_SIZE])
        feed_dict[init_memory_output] = np.zeros([valid_size, HIDDEN_SIZE])

    else: # test
        for i in range(SEQ_LENGTH + 1):
            feed_dict[data_buffers[i]] = test_batch[i]
        
        # Set previous weights of read and write heades.
        for i in range(SEQ_LENGTH):
            feed_dict[prev_read_weights_seq[i]] = prev_rw_seq[i]
            feed_dict[prev_update_weights_seq[i]] = prev_uw_seq[i]

        # Reset "init" state and output of controller.
        feed_dict[init_controller_output] = np.zeros([test_size, HIDDEN_SIZE])
        feed_dict[init_controller_state] = np.zeros([test_size, HIDDEN_SIZE])
        feed_dict[init_memory_output] = np.zeros([test_size, HIDDEN_SIZE])
       
    return feed_dict

print("Feed_dict definition OK")

Feed_dict definition OK


### Session execution

In [13]:
# Eventually clear the log dir.
if tf.gfile.Exists(LOG_DIR):
  tf.gfile.DeleteRecursively(LOG_DIR)
# Create (new) log dir.
tf.gfile.MakeDirs(LOG_DIR)

print("Log dir CLEARED")

Log dir CLEARED


In [14]:

# Create session to execute graph.
sess=tf.InteractiveSession()

# Create summary writers, point them to LOG_DIR.
train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
valid_writer = tf.summary.FileWriter(LOG_DIR + '/valid')
test_writer = tf.summary.FileWriter(LOG_DIR + '/test')

# Initialize global variables.
tf.global_variables_initializer().run()
print('Variables initialized')


# Create initial previous read and update - full of zeros. 
prev_rw_seq = list()
prev_uw_seq = list()
for i in range(SEQ_LENGTH):
    prev_rw_seq.append(np.zeros([1, MEMORY_SLOTS]))
    prev_uw_seq.append(np.zeros([1, MEMORY_SLOTS]))

# Determine how long to perform the training and how often the test loss on validation batch will be computed. 
summary_frequency = 100
validation_frequency = 1000
#num_steps = 20 #// (BATCH_SIZE*SEQ_LENGTH)
num_steps = train_size // (BATCH_SIZE*SEQ_LENGTH)
print("Number of iterations per epoch =", num_steps)
for step in range(num_steps):
    input_seq_batch_, memory_, memory_update_seq_, prev_rw_seq, prev_uw_seq, summaries, _, loss_, lr_ = sess.run([
        input_seq_batch, memory, memory_update_seq, read_weights_seq, update_weights_seq, merged_summaries, optimizer, loss, learning_rate],
        feed_dict=create_feed_dict("train"))
    # Every (100) steps collect statistics.
    if step % summary_frequency == 0:
        # Add summary.
        train_writer.add_summary(summaries, step*BATCH_SIZE*SEQ_LENGTH)
        train_writer.flush()
        print('Training set BPC at step %d: %0.5f learning rate: %f' % (step, loss_, lr_))

        #print("memory=\n", memory_)
        #print("memory_update_seq=\n", memory_update_seq_)
        # Print loss from last batch.
    
    #if step % validation_frequency == 0:
    #    # Validation set BPC.
    #    print('=' * 80)
    #    print("Calculating BPC on validation set")
    #    v_summaries, v_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("valid"))
    #    print("Validation set BPC: %.5f" % v_loss)
    #    valid_writer.add_summary(v_summaries, step*BATCH_SIZE*SEQ_LENGTH)
    #    valid_writer.flush()
    # End of statistics collection

for gv in grads_and_vars:
    #print(str(sess.run(gv[0])) + " - " + gv[1].name)        
    print(" - " + gv[1].name)        

# Test set BPC.
#print('=' * 80)
#print("Calculating BPC on test set")
#t_summary, t_loss = sess.run([merged_summaries, loss], feed_dict=create_feed_dict("test"))
#print("Final test set BPC: %.5f" % t_loss)
#test_writer.add_summary(t_summary, step*BATCH_SIZE*SEQ_LENGTH)
#test_writer.flush()
    
# Close writers and session.
train_writer.close()
valid_writer.close()
test_writer.close()
sess.close() 

Variables initialized
Number of iterations per epoch = 510161
Training set BPC at step 0: 4.17746 learning rate: 0.100000
Training set BPC at step 100: 2.67705 learning rate: 0.100000
Training set BPC at step 200: 4.51366 learning rate: 0.100000
Training set BPC at step 300: 3.21168 learning rate: 0.100000
Training set BPC at step 400: 4.96880 learning rate: 0.100000
Training set BPC at step 500: 3.17881 learning rate: 0.100000
Training set BPC at step 600: 4.11189 learning rate: 0.100000
Training set BPC at step 700: 5.24490 learning rate: 0.100000
Training set BPC at step 800: 3.36635 learning rate: 0.100000
Training set BPC at step 900: 4.73788 learning rate: 0.100000
Training set BPC at step 1000: 2.03898 learning rate: 0.100000
Training set BPC at step 1100: 1.90097 learning rate: 0.100000
Training set BPC at step 1200: 3.28844 learning rate: 0.100000
Training set BPC at step 1300: 1.42545 learning rate: 0.100000
Training set BPC at step 1400: 4.02794 learning rate: 0.100000
Train

Training set BPC at step 12900: 4.16535 learning rate: 0.081000
Training set BPC at step 13000: 3.21622 learning rate: 0.081000
Training set BPC at step 13100: 5.82554 learning rate: 0.081000
Training set BPC at step 13200: 3.70456 learning rate: 0.081000
Training set BPC at step 13300: 2.49387 learning rate: 0.081000
Training set BPC at step 13400: 4.40847 learning rate: 0.081000
Training set BPC at step 13500: 4.60051 learning rate: 0.081000
Training set BPC at step 13600: 2.59548 learning rate: 0.081000
Training set BPC at step 13700: 4.98207 learning rate: 0.081000
Training set BPC at step 13800: 2.62246 learning rate: 0.081000
Training set BPC at step 13900: 3.01380 learning rate: 0.081000
Training set BPC at step 14000: 2.21154 learning rate: 0.081000
Training set BPC at step 14100: 4.48807 learning rate: 0.081000
Training set BPC at step 14200: 2.36017 learning rate: 0.081000
Training set BPC at step 14300: 3.53130 learning rate: 0.081000
Training set BPC at step 14400: 2.34457 

Training set BPC at step 25800: 2.88933 learning rate: 0.059049
Training set BPC at step 25900: 2.28722 learning rate: 0.059049
Training set BPC at step 26000: 2.42170 learning rate: 0.059049
Training set BPC at step 26100: 3.49465 learning rate: 0.059049
Training set BPC at step 26200: 3.30808 learning rate: 0.059049
Training set BPC at step 26300: 2.21268 learning rate: 0.059049
Training set BPC at step 26400: 3.37240 learning rate: 0.059049
Training set BPC at step 26500: 2.75278 learning rate: 0.059049
Training set BPC at step 26600: 3.20420 learning rate: 0.059049
Training set BPC at step 26700: 3.59442 learning rate: 0.059049
Training set BPC at step 26800: 4.58408 learning rate: 0.059049
Training set BPC at step 26900: 3.08925 learning rate: 0.059049
Training set BPC at step 27000: 2.95812 learning rate: 0.059049
Training set BPC at step 27100: 2.43039 learning rate: 0.059049
Training set BPC at step 27200: 2.50240 learning rate: 0.059049
Training set BPC at step 27300: 3.59736 

KeyboardInterrupt: 